In [5]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import MaxPool2D

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [152]:
model =Sequential()

In [153]:
model.add(Conv2D(32,(3,3),input_shape=(48,48,3),activation="relu"))

In [154]:
model.add(MaxPool2D(pool_size=(2,2)))

In [155]:
model.add(Conv2D(64,(3,3),activation="relu"))

In [156]:
model.add(MaxPool2D(pool_size=(2,2)))

In [157]:
model.add(Flatten())

In [158]:
model.add(Dense(units=128,activation="relu"))
model.add(Dense(units=128,activation="relu"))
model.add(Dense(units=64,activation="relu"))
model.add(Dense(units=7,activation="softmax"))

In [159]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [160]:
from keras.preprocessing.image import ImageDataGenerator

In [161]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [162]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [163]:
training_set = train_datagen.flow_from_directory('Training',
                                                 target_size = (48, 48),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 28708 images belonging to 7 classes.


In [164]:

valid_set = test_datagen.flow_from_directory('PrivateTest',
                                            target_size = (48, 48),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 3589 images belonging to 7 classes.


In [165]:
test_set = test_datagen.flow_from_directory('public',
                                            target_size = (48, 48),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 0 images belonging to 0 classes.


In [ ]:
import PIL.Image
from PIL.Image import Image
#from keras.preprocessing import image

In [ ]:
model.fit_generator(training_set,steps_per_epoch=28708,epochs=3, validation_data=valid_set,validation_steps=3589)

Epoch 1/3
28708/28708 [==============================] - 4764s 166ms/step - loss: 1.0726 - acc: 0.5927 - val_loss: 1.2656 - val_acc: 0.5702
Epoch 2/3
28708/28708 [==============================] - 5071s 177ms/step - loss: 0.7430 - acc: 0.7261 - val_loss: 1.3936 - val_acc: 0.5687
Epoch 3/3
25757/28708 [=========================>....] - ETA: 8:40 - loss: 0.6091 - acc: 0.7785

In [ ]:
prediction=model.predict_generator(test_set)

In [ ]:
pred=[]
for i in range(prediction.shape[0]):
    pred.append(list(prediction[i]).index(max(prediction[i])))

In [ ]:
import pandas as pd
actual=pd.read_csv("private_data.csv")

In [ ]:
for i,j in zip(actual["emotion"],pred):
    print(i,j)

In [ ]:
actual_values=[]
for i in range(3589):
    if i<467:
        actual_values.append(0)
    elif i<523:
        actual_values.append(1)
    elif i<1019:
        actual_values.append(2)
    elif i<1914:
        actual_values.append(3)
    elif i<2567:
        actual_values.append(4)
    elif i<2982:
        actual_values.append(5)
    elif i<3589:
        actual_values.append(6)

In [ ]:
import pandas as pd
from keras.preprocessing import image
import numpy as np

In [ ]:
pred_data=pd.read_csv("PublicTestData.csv")

In [ ]:
path=r"PublicTest"
result=[]
for i in range(3589):
    img=image.load_img(os.path.join(path,pred_data.iloc[i,3]),target_size=(48,48))
    img=image.img_to_array(img)
    img=np.expand_dims(img,axis=0)
    res=list(model.predict(img)[0])
    result.append(res.index(max(res)))

In [ ]:
from sklearn.metrics import precision_score
print(precision_score(pred_data["Emotion"],result,average="weighted"))